In [ ]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [ ]:
from groq import Groq

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Initialize Groq client
client = Groq(api_key='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')  # Use your Groq API key

def query_tablellm(csv_data, question, max_tokens=400):
    # Create the prompt following the TableLLM format for code solution generation
    prompt = f"""[INST]You are provided with the first few lines of a CSV file. Using the data and columns shown, write a Python program to solve the specified problem.
    The program should store the final answer in a variable named result.

Make sure to write only the Python code, and it should be enclosed in triple quotes (\"\"\" \"\"\"). Do not use triple quotes elsewhere.

Key Requirements:

Data Input: Do not include any code for reading the CSV file (e.g., df = pd.read_csv('data.csv')). Assume the data has already been loaded into a DataFrame named df.
Output: The final result must be stored in the result variable, without printing or returning it.
The python should not contain packages other than : allowed_packages = {"numpy", "pandas" , "sklearn"}
Only use import sklearn. Do not import anything from sklearn.
Please ensure that all functions, classes, and methods from  sklearn are used in their fully qualified form.
For example, instead of directly importing LabelEncoder, use sklearn.preprocessing.LabelEncoder.

Do not use from ... import ... statements for individual functions or classes. For instance:
Use sklearn.model_selection.train_test_split instead of importing train_test_split.

Header and first few lines of CSV file:
{csv_data}

Question: {question}[/INST]"""

    # Send prompt to Groq API
    response = client.chat.completions.create(
        model="llama-3.1-70b-versatile",  # Specify the model you want to use
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.7,         # Control randomness
        max_tokens=max_tokens,   # Control maximum tokens in the output
        top_p=0.9,               # Sampling control for nucleus sampling
        stream=False             # Change to True if you want streaming responses
    )

    # Access the content of the first choice in the response
    result = response.choices[0].message.content
    return result



# Example table data (in CSV format)
csv_data = """Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
40,M,ATA,140,289,0,Normal,172,N,0.0,Up,no
49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,yes
37,M,ATA,130,283,0,ST,98,N,0.0,Up,no
48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,yes"""

# Example question about the table
question = "Which factor has highest correlation with Heart Disease ?"

# Get the answer from the TableLLM model
response = query_tablellm(csv_data, question)

# Print the model's response
print("Generated Answer:\n", response)



Generated Answer:
 """
import pandas as pd
import numpy as np

# Assuming df is the DataFrame containing the data

# Convert categorical variables to numerical
df['Sex'] = df['Sex'].map({'M': 0, 'F': 1})
df['ChestPainType'] = df['ChestPainType'].map({'ATA': 0, 'NAP': 1, 'ASY': 2, 'TA': 3})
df['RestingECG'] = df['RestingECG'].map({'Normal': 0, 'ST': 1, 'LVH': 2})
df['ExerciseAngina'] = df['ExerciseAngina'].map({'N': 0, 'Y': 1})
df['ST_Slope'] = df['ST_Slope'].map({'Up': 0, 'Flat': 1, 'Down': 2})
df['HeartDisease'] = df['HeartDisease'].map({'no': 0, 'yes': 1})

# Calculate correlation matrix
corr_matrix = df.corr()

# Get the column with the highest correlation with Heart Disease
result = corr_matrix['HeartDisease'].abs().sort_values(ascending=False).index[1]
"""


In [ ]:
import re
def extract_python_code(text):
    # Regular expression to capture everything inside triple quotes
    pattern = r'"""(.*?)"""'

    # Find the content inside the triple quotes
    match = re.search(pattern, text, re.DOTALL)

    if match:
        # Return the code inside the triple quotes
        return match.group(1).strip()
    else:
        return "No content found inside triple quotes."
python_code = extract_python_code(response)


In [ ]:
python_code

"import pandas as pd\nimport numpy as np\n\n# Assuming df is the DataFrame containing the data\n\n# Convert categorical variables to numerical\ndf['Sex'] = df['Sex'].map({'M': 0, 'F': 1})\ndf['ChestPainType'] = df['ChestPainType'].map({'ATA': 0, 'NAP': 1, 'ASY': 2, 'TA': 3})\ndf['RestingECG'] = df['RestingECG'].map({'Normal': 0, 'ST': 1, 'LVH': 2})\ndf['ExerciseAngina'] = df['ExerciseAngina'].map({'N': 0, 'Y': 1})\ndf['ST_Slope'] = df['ST_Slope'].map({'Up': 0, 'Flat': 1, 'Down': 2})\ndf['HeartDisease'] = df['HeartDisease'].map({'no': 0, 'yes': 1})\n\n# Calculate correlation matrix\ncorr_matrix = df.corr()\n\n# Get the column with the highest correlation with Heart Disease\nresult = corr_matrix['HeartDisease'].abs().sort_values(ascending=False).index[1]"

In [ ]:
import copy
import numpy as np
#from .preprocessing import convert_categorical_to_integer_f
from typing import Any, Dict, Optional
import pandas as pd

def run_llm_code(code: str, df: pd.DataFrame, convert_categorical_to_integer: Optional[bool] = True, fill_na: Optional[bool] = True):
    """
    Executes the given code on the given dataframe and returns the resulting dataframe.

    Parameters:
    code (str): The code to execute.
    df (pandas.DataFrame): The dataframe to execute the code on.
    convert_categorical_to_integer (bool, optional): Whether to convert categorical columns to integer values. Defaults to True.
    fill_na (bool, optional): Whether to fill NaN values in object columns with empty strings. Defaults to True.

    Returns:
    pandas.DataFrame: The resulting dataframe after executing the code.
    """
    try:
        loc = {}
        df = copy.deepcopy(df)

        # if fill_na and False:
        #     df.loc[:, (df.dtypes == object)] = df.loc[:, (df.dtypes == object)].fillna(
        #         ""
        #     )
        # if convert_categorical_to_integer and False:
        #     df = df.apply(convert_categorical_to_integer_f)

        access_scope = {"df": df, "pd": pd, "np": np}
        parsed = ast.parse(code)
        check_ast(parsed)
        exec(compile(parsed, filename="<ast>", mode="exec"), access_scope, loc)
        df = copy.deepcopy(df)

    except Exception as e:
        print("Code could not be executed", e)
        raise (e)

    return df , loc


import ast
import pandas as pd


def check_ast(node: ast.AST) -> None:
    """
    Checks if the given AST node is allowed.

    Parameters:
    node (ast.AST): The AST node to check.

    Raises:
    ValueError: If the AST node is not allowed.
    """
    allowed_nodes = {
        ast.Module,
        ast.Expr,
        ast.Load,
        ast.BinOp,
        ast.UnaryOp,
        ast.Add,
        ast.Sub,
        ast.Mult,
        ast.Div,
        ast.FloorDiv,
        ast.Mod,
        ast.Pow,
        ast.USub,
        ast.UAdd,
        ast.Num,
        ast.Str,
        ast.Bytes,
        ast.List,
        ast.Tuple,
        ast.Dict,
        ast.Name,
        ast.Call,
        ast.Attribute,
        ast.keyword,
        ast.Subscript,
        ast.Index,
        ast.Slice,
        ast.ExtSlice,
        ast.Assign,
        ast.AugAssign,
        ast.NameConstant,
        ast.Compare,
        ast.Eq,
        ast.NotEq,
        ast.Lt,
        ast.LtE,
        ast.Gt,
        ast.GtE,
        ast.Is,
        ast.IsNot,
        ast.In,
        ast.NotIn,
        ast.And,
        ast.Or,
        ast.BitOr,
        ast.BitAnd,
        ast.BitXor,
        ast.Invert,
        ast.Not,
        ast.Constant,
        ast.Store,
        ast.If,
        ast.IfExp,
        # These nodes represent loop structures. If you allow arbitrary loops, a user could potentially create an infinite loop that consumes system resources and slows down or crashes your system.
        ast.For,
        ast.While,
        ast.Break,
        ast.Continue,
        ast.Pass,
        ast.Assert,
        ast.Return,
        ast.FunctionDef,
        ast.ListComp,
        ast.SetComp,
        ast.DictComp,
        ast.GeneratorExp,
        ast.Await,
        # These nodes represent the yield keyword, which is used in generator functions. If you allow arbitrary generator functions, a user might be able to create a generator that produces an infinite sequence, potentially consuming system resources and slowing down or crashing your system.
        ast.Yield,
        ast.YieldFrom,
        ast.Lambda,
        ast.BoolOp,
        ast.FormattedValue,
        ast.JoinedStr,
        ast.Set,
        ast.Ellipsis,
        ast.expr,
        ast.stmt,
        ast.expr_context,
        ast.boolop,
        ast.operator,
        ast.unaryop,
        ast.cmpop,
        ast.comprehension,
        ast.arguments,
        ast.arg,
        ast.Import,
        ast.ImportFrom,
        ast.alias,
    }

    allowed_packages = {"numpy", "pandas", "sklearn"}

    allowed_funcs = {
        "sum": sum,
        "min": min,
        "max": max,
        "abs": abs,
        "round": round,
        "len": len,
        "str": str,
        "int": int,
        "float": float,
        "bool": bool,
        "list": list,
        "dict": dict,
        "set": set,
        "tuple": tuple,
        "enumerate": enumerate,
        "zip": zip,
        "range": range,
        "sorted": sorted,
        "reversed": reversed,
        "print":print,


        # Add other functions you want to allow here.
    }

    allowed_attrs = {
        # NP
        "array",
        "arange",
        "values",
        "linspace",
        # PD
        "mean",
        "sum",
        "contains",
        "where",
        "min",
        "max",
        "median",
        "std",
        "sqrt",
        "pow",
        "iloc",
        "cut",
        "qcut",
        "inf",
        "nan",
        "isna",
        "map",
        "reshape",
        "shape",
        "split",
        "var",
        "codes",
        "abs",
        "cumsum",
        "cumprod",
        "cummax",
        "cummin",
        "diff",
        "repeat",
        "index",
        "log",
        "log10",
        "log1p",
        "slice",
        "exp",
        "expm1",
        "pow",
        "pct_change",
        "corr",
        "cov",
        "round",
        "clip",
        "dot",
        "transpose",
        "T",
        "astype",
        "copy",
        "drop",
        "dropna",
        "fillna",
        "replace",
        "merge",
        "append",
        "join",
        "groupby",
        "resample",
        "rolling",
        "expanding",
        "ewm",
        "agg",
        "aggregate",
        "filter",
        "transform",
        "apply",
        "pivot",
        "melt",
        "sort_values",
        "sort_index",
        "reset_index",
        "set_index",
        "reindex",
        "shift",
        "extract",
        "rename",
        "tail",
        "head",
        "describe",
        "count",
        "value_counts",
        "unique",
        "nunique",
        "idxmin",
        "idxmax",
        "isin",
        "between",
        "duplicated",
        "rank",
        "to_numpy",
        "to_dict",
        "to_list",
        "to_frame",
        "squeeze",
        "add",
        "sub",
        "mul",
        "div",
        "mod",
        "columns",
        "loc",
        "lt",
        "le",
        "eq",
        "ne",
        "ge",
        "gt",
        "all",
        "any",
        "clip",
        "conj",
        "conjugate",
        "round",
        "trace",
        "cumprod",
        "cumsum",
        "prod",
        "dot",
        "flatten",
        "ravel",
        "T",
        "transpose",
        "swapaxes",
        "clip",
        "item",
        "tolist",
        "argmax",
        "argmin",
        "argsort",
        "max",
        "mean",
        "min",
        "nonzero",
        "ptp",
        "sort",
        "std",
        "var",
        "str",
        "dt",
        "cat",
        "sparse",
        "plot",
        "subset",
        "get_loc",
        "DataFrame",
        "get_dummies",
        # Add other DataFrame methods you want to allow here.

        # Model Selection
    "sklearn.model_selection.train_test_split",
    "sklearn.model_selection.cross_val_score",
    "sklearn.model_selection.GridSearchCV",
    "sklearn.model_selection.RandomizedSearchCV",
    "sklearn.model_selection.StratifiedKFold",
    "sklearn.model_selection.KFold",

    # Metrics
    "sklearn.metrics.accuracy_score",
    "sklearn.metrics.precision_score",
    "sklearn.metrics.recall_score",
    "sklearn.metrics.f1_score",
    "sklearn.metrics.roc_auc_score",
    "sklearn.metrics.confusion_matrix",
    "sklearn.metrics.classification_report",
    "sklearn.metrics.mean_squared_error",
    "sklearn.metrics.r2_score",
    "sklearn.metrics.adjusted_rand_score",
    "sklearn.metrics.silhouette_score",

    # Preprocessing
    "sklearn.preprocessing.StandardScaler",
    "sklearn.preprocessing.MinMaxScaler",
    "sklearn.preprocessing.Normalizer",
    "sklearn.preprocessing.Binarizer",
    "sklearn.preprocessing.LabelEncoder",
    "sklearn.preprocessing.OneHotEncoder",
    "sklearn.preprocessing.PolynomialFeatures",
    "sklearn.preprocessing.RobustScaler",

    # Decomposition
    "sklearn.decomposition.PCA",
    "sklearn.decomposition.TruncatedSVD",

    # Clustering
    "sklearn.cluster.KMeans",
    "sklearn.cluster.DBSCAN",
    "sklearn.cluster.AgglomerativeClustering",

    # Regression
    "sklearn.linear_model.LinearRegression",
    "sklearn.linear_model.Ridge",
    "sklearn.linear_model.Lasso",
    "sklearn.linear_model.ElasticNet",
    "sklearn.linear_model.LogisticRegression",
    "sklearn.svm.SVR",
    "sklearn.tree.DecisionTreeRegressor",
    "sklearn.ensemble.RandomForestRegressor",

    # Classification
    "sklearn.svm.SVC",
    "sklearn.tree.DecisionTreeClassifier",
    "sklearn.ensemble.RandomForestClassifier",
    "sklearn.ensemble.GradientBoostingClassifier",
    }

    # Check if the node type is allowed
    if type(node) not in allowed_nodes:
        raise ValueError(f"Disallowed code: {ast.unparse(node)} is {type(node)}")

    # Check function calls
    if isinstance(node, ast.Call) and isinstance(node.func, ast.Name):
        if node.func.id not in allowed_funcs:
            # Try checking if the function exists in the current environment
            if not hasattr(__builtins__, node.func.id) and not is_function_imported(node.func.id):
                raise ValueError(f"Disallowed function: {node.func.id}")

    # Check attributes
    if isinstance(node, ast.Attribute):
        if node.attr not in allowed_attrs:
            # Try checking if the attribute exists on the object
            if not hasattr(getattr(__builtins__, node.attr, None), node.attr):
                raise ValueError(f"Disallowed attribute: {node.attr}")

    # Check imports
    if isinstance(node, ast.Import) or isinstance(node, ast.ImportFrom):
        for alias in node.names:
            if alias.name not in allowed_packages:
                # Try importing the package to see if it exists
                if not package_exists(alias.name):
                    raise ValueError(f"Disallowed package import: {alias.name}")

    # Recursively check child nodes
    for child in ast.iter_child_nodes(node):
        check_ast(child)

def package_exists(package_name):
    try:
        importlib.import_module(package_name)
        return True
    except ImportError:
        return False

def is_function_imported(func_name):
    # Check if function is part of the environment (imported from a package)
    try:
        return func_name in globals() or func_name in locals()
    except KeyError:
        return False


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the TableLLM-13B model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("RUCKBReasoning/TableLLM-7b" )
model = AutoModelForCausalLM.from_pretrained("RUCKBReasoning/TableLLM-7b")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Now we are given the response. We have to now execute the code and return the answer
df = pd.read_csv('/content/drive/MyDrive/CAAFE/heart.csv')
df , loc = run_llm_code(python_code,df)
final_ans = loc['result']
max_tokens = 200

prompt2 = (
    f"Here is a question and its corresponding answer:\n\n"
    f"Question: {question}\n\n"
    f"Answer: {final_ans}\n\n"
    "Using the provided answer, write a clear and concise sentence that directly addresses the question. "
    "Ensure that the response is a straightforward statement that presents the final answer without additional explanations or calculations."
)



# Tokenize the prompt
inputs = tokenizer(prompt2, return_tensors="pt", truncation=True, padding=True)

# Move inputs to the same device as the model
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate the response from the model
output = model.generate(
      inputs["input_ids"],
      max_new_tokens=max_tokens,
      temperature=0.5,  # Adjust this if needed for randomness
      top_p=0.9,
      do_sample=True
)
# Decode the output tokens back into text
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/893 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Here is a question and its corresponding answer:

Question: Which factor has highest correlation with Heart Disease ?

Answer: ST_Slope

Using the provided answer, write a clear and concise sentence that directly addresses the question. Ensure that the response is a straightforward statement that presents the final answer without additional explanations or calculations.

The factor with the highest correlation with Heart Disease, as per the provided table, is "ST_Slope."
